In [3]:
import tensorflow as tf
import numpy as np
from nets import inception_resnet_v1
from nets.inception_resnet_v1 import *
from nets.mobilenet_v2 import *
from nets.wynet import *
from preprocessing import face_preprocessing
from tensorflow.contrib import slim
import urllib2
import math
from scipy import misc
from preprocessing import inception_preprocessing

IMAGE_SIZE = 256
CHANNELS = 3
FEATURE_DIM = 512   #1792


filelist = '/home/zeng/workspace/tensorflow-workspace/extra_id256.txt'

image_paths = [l.strip('\n') for l in
    tf.gfile.FastGFile(filelist, 'r').readlines()]
batch_size = 60
tf.reset_default_graph()

def go_tf(image_paths):
    nrof_samples = len(image_paths)
    images = np.zeros((nrof_samples, IMAGE_SIZE, IMAGE_SIZE, 3))
    for i in range(nrof_samples):
        img = misc.imread(image_paths[i])
        images[i,:,:,:] = img
        
    return images


def preproc_image_batch(batch_size, num_threads=1):
    
    
    # Read image file from disk and decode JPEG
    filename_queue = tf.FIFOQueue(100000, [tf.string], shapes=[[]], name="filename_queue")
    reader = tf.WholeFileReader()
    image_filename, image_raw = reader.read(filename_queue)
    image = tf.image.decode_jpeg(image_raw, channels=3)
    # Image preprocessing
    #preproc_func_name = self._network_name if self._preproc_func_name is None else self._preproc_func_name
    #image_preproc_fn = preprocessing_factory.get_preprocessing(preproc_func_name, is_training=False)
    #image_preproc = image_preproc_fn(image, self.image_size, self.image_size)
    image_preproc = face_preprocessing.preprocess_image(image, IMAGE_SIZE, IMAGE_SIZE, is_training=False)
    #image_preproc  = tf.expand_dims(image_preproc, 0)
    # Read a batch of preprocessing images from queue
    image_batch = tf.train.batch(
        [image_preproc, image_filename], batch_size, num_threads=num_threads,
        allow_smaller_final_batch=True)
    return image_batch


    
with tf.Graph().as_default():
    
   
    #pl_image_files = tf.placeholder(tf.string, shape=[None], name="image_file_list")
    #enqueue_op = filename_queue.enqueue_many([pl_image_files])
    
    batch_from_queue, _batch_filenames = preproc_image_batch(batch_size, num_threads=1)
    image_batch = tf.placeholder_with_default(
        batch_from_queue, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 3])
    #with slim.arg_scope(inception_resnet_v1_arg_scope()):
    #    logits, end_points = inception_resnet_v1(image_batch)
    with slim.arg_scope(mobilenet_v2_arg_scope()):
        logits, end_points = mobilenet_v2_075(image_batch, num_classes=10572)
    #variables_to_restore = slim.get_variables_to_restore()

    
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    fetches = {}
    #fetches['PreLogitsFlatten'] = end_points['PreLogitsFlatten']
    #fetches['fc5'] = end_points['fc5']
    fetches['AvgPool_1a'] = end_points['AvgPool_1a']
    
    
    with tf.Session() as sess:
         
        init_fn = slim.assign_from_checkpoint_fn(
            '/data/zeng/ft_0.01/model.ckpt-15000',slim.get_model_variables('MobilenetV2'))
        init_fn(sess)
        #graph = sess.graph
        #print([node.name for node in graph.as_graph_def().node])
        
        
        
        #sess.run(tf.local_variables_initializer())
        nrof_images = len(image_paths)
        nrof_batches = int(math.ceil(1.0*nrof_images / batch_size))
        feature_batch = np.zeros((nrof_images, FEATURE_DIM))
        for i in range(nrof_batches):
            start_index = i * batch_size
            end_index = min((i+1)*batch_size, nrof_images)
            paths_batch = image_paths[start_index:end_index]
            #
            input_batch = go_tf(paths_batch)
            #
            feed_dict = {image_batch: input_batch}
            outputs = sess.run(fetches, feed_dict=feed_dict)
            feature_batch[start_index:end_index,:] = np.squeeze( outputs['AvgPool_1a'] )
            
        np.save('/home/zeng/workspace/metric_results_/0.01_extra', feature_batch)
        print('done')
    




INFO:tensorflow:Restoring parameters from /data/zeng/ft_0.01/model.ckpt-15000


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /data/zeng/ft_0.01/model.ckpt-15000
	 [[Node: save/RestoreV2_123 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_123/tensor_names, save/RestoreV2_123/shape_and_slices)]]
	 [[Node: save/RestoreV2_241/_179 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_716_save/RestoreV2_241", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save/RestoreV2_123', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    ap

In [2]:
x = outputs['PreLogitsFlatten']
x.shape
type(x)

KeyError: 'PreLogitsFlatten'

In [ ]:
x=graph.as_graph_def().node

In [ ]:
x.name